In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Imported CSV files to dataframes

In [ ]:
incident_response = pd.read_csv('../Data/charts_clean.csv')
incident_response.head()

In [ ]:
calls_address = pd.read_csv('../Data/calls_clean.csv')
calls_address.head()

In [ ]:
geo_clean = pd.read_csv('../Data/geocodeclean.csv')
geo_clean.head()

## Merged geo data to calls/address

In [ ]:
geo_calls_merged = pd.merge(geo_clean, calls_address, left_on = 'address', right_on = 'Address')
geo_calls_merged.drop(['address'], axis=1, inplace = True)
geo_calls_merged.head()

## Consolidated priority calls and dropped excess priority columns

In [ ]:
incident_response['Priority_2'] = incident_response['Priority_2'] + incident_response['Priority_2L'] + incident_response['Priority_2M'] + incident_response['Priority_2N'] + incident_response['Priority_2T'] + incident_response['Priority_2R']
incident_response['Priority_3'] = incident_response['Priority_3'] + incident_response['Priority_3H'] + incident_response['Priority_3M'] + incident_response['Priority_3Q']
incident_response.drop(['Priority_2L', 'Priority_2M', 'Priority_2N', 'Priority_2T', 'Priority_2R', 'Priority_3H', 'Priority_3M', 'Priority_3Q'], axis=1, inplace = True)
incident_response.head()